## Table of Contents
* [Setup](#Setup)
	* [estimate cost](#estimate-cost)
	* [load dataset](#load-dataset)
* [Submitting HITs](#Submitting-HITs)
* [Retrieve results](#Retrieve-results)
* [Interact with workers](#Interact-with-workers)
* [Accepting and deleting HITs... careful with these](#Accepting-and-deleting-HITs...-careful-with-these)


In [45]:
sandbox = False

# sent_file = "/Users/nikett/quick/sciencesrl-annotation-afresh/inputs/batch-8ksubsetof15k-highconf.tsv"
# sent_file = "/Users/nikett/quick/sciencesrl-annotation-afresh/inputs/gold_testset_input.tsv"
# sent_file = "/Users/nikett/quick/sciencesrl-annotation-afresh/inputs/sample_input.tsv"
# sent_file = "/Users/nikett/quick/sciencesrl-annotation-afresh/inputs/v5_3.onlyagreedverbs.tsv"
sent_file = "/Users/nikett/quick/sciencesrl-annotation-afresh/inputs/v5_3.onlyagreedverbs__round3.tsv"

num_sentences = 1  # -1 indicates take all sentences, anynumber >0 indicates restriction
outfile='/Users/nikett/quick/sciencesrl-annotation-afresh/results/batch_v5_3__round3.pkl' if not sandbox else '/Users/nikett/quick/sciencesrl-annotation-afresh/results/sandbox_v2_sample.pkl'

# CAREFUL HERE
# outfile = '/Users/nikett/quick/science68k/predicate_id/results/batch_v4_big2_and_v3_finalversion.pkl'

static_params = {
    'title': "Answer simple questions about actions in the sentence.",
    'description': "Answer simple questions about actions in the sentence.",
    'keywords': ['English verbs'],
    'frame_height': 1000,
    'amount': 0.10,  # CAREFUL HERE
    'duration': 3600 * 1,
    'lifetime': 3600 * 24 * 2,
    'max_assignments': 1 if sandbox else 3,
    'locales': ['US', 'CA', 'AU', 'NZ', 'GB']
}

In [46]:
import numpy as np
import pandas as pd
import re
import pickle
import boto
import json
import os
import random
from copy import deepcopy
from tqdm import tqdm

from IPython.core.display import HTML
from mturk_utils.annotation_collection import prepare_hit
from mturk_utils.annotation_collection import write_task_page
from mturk_utils.mturk import expected_cost
import mturk_ai2
from mturk_utils.mturk import MTurk

# boto.set_stream_logger('boto')

# Submitting HITs

In [52]:
turk_account = mturk_ai2
rw_host='mechanicalturk.amazonaws.com'
amt_con = MTurk(turk_account.access_key, turk_account.access_secret_key) if sandbox else MTurk(turk_account.access_key, turk_account.access_secret_key, host=rw_host)
amt_con.get_account_balance()

$382.55

In [38]:
df = pd.read_csv(sent_file, sep='\t', header=0)
print "First few inputs of", len(df) ,"sentences ..."
df.head(5)

First few inputs of 1744 sentences ...


,sentid,sent,verbids
0,52658,The consumer advocacy magazine found that the ...,"4-5,9-10,14-15,19-20"
1,56693,"If an eclipse happens during this situation , ...","3-4,10-12,20-21,27-28"
2,52799,Cooling or compressing a vapor causes it to sh...,"0-1,2-3,8-9,23-24"
3,47795,The amount of water that evaporates from the l...,"5-6,16-17"
4,45153,"When the ice has melted into water , you can a...","4-5,10-11,17-18,19-20"


In [39]:
def removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

build_hit_group_with_empties = [prepare_hit(row.sentid, removeNonAscii(str.strip(row.sent)), row.verbids, static_params) for index, row in (df.head(num_sentences).iterrows() if num_sentences > 0 else df.iterrows())]
print "Debug: #hits with empties = ", len(build_hit_group_with_empties)
build_hit_group = [x for x in build_hit_group_with_empties if x is not None]
print "Debug: #hits without empties = ", len(build_hit_group)
write_task_page(random.choice(build_hit_group)['html'])
expected_cost(build_hit_group, static_params, amt_con)

Debug: #hits with empties =  1744
Debug: #hits without empties =  1744


523.2

In [40]:
hit_group = [amt_con.create_html_hit(single_hit) for single_hit in tqdm(build_hit_group)]

100%|██████████| 1744/1744 [06:14<00:00,  5.04it/s]


# Retrieve results

In [ ]:
#from mturk_utils.annotation_collection import create_result
from mturk_utils.annotation_collection import get_assignments
from mturk_utils.annotation_collection import pickle_this
from mturk_utils.annotation_collection import filter_hits_by_date

# number 16000-17000 onwards starts the actual  hits for scisrl part2, before that predid, after 16K 
# Note: 6901 hits until page 499 (i.e. pages 164,250 with page_size=100)
# 16400 onwards sciscrl version2 
barebone_hits = []
for page_num in range(164,3000):
    print "fetching page ", page_num, " ..."
    barebone_hits += [hit for hit in amt_con.connection.get_reviewable_hits(page_size=100, page_number=page_num)]
print "Now fetching detailed HITs for ", len(barebone_hits), "..."
detailed_hits = [amt_con.connection.get_hit(hit.HITId, response_groups=('Minimal','HITDetail'))[0]
                             for hit in tqdm(barebone_hits)]

# for hit in detailed_hits: print hit.HITId, hit.Title
# See: http://boto3.readthedocs.io/en/latest/reference/services/mturk.html?highlight=gethit
# for all fields such as Title and creation time.

#FIXME How to find hits that are not yet approved or rejected? That is, have not been worked upon by anyone.

# predid: Mark actions in a sentence.
# scisrl v2 (8k) of updated big task: Answer simple questions about actions in a sentence. ("the actions" -> "actions")
# the above was before page 165 (size 100 per page.)
# scisrl v1 (300) of updated big task: Answer simple questions about the actions in a sentence.
# scisrl v1 (8k-v5_3--round1) of updated big task: Answer simple questions about the actions in a sentence.
# scisrl v1 (8k-v5_3--round2) of updated big task: Answer simple questions about the actions in a sentence
# scisrl v1 (8k-v5_3--round3) of updated big task: Answer simple questions about actions in the sentence.
hits_to_serialize = [hit for hit in detailed_hits if str(hit.Title)=='Answer simple questions about actions in the sentence.']

print "Number of Hits to serialize = ", len(hits_to_serialize), "\nNow fetching assignments..."

# FIXME uncomment this.
# all_hits = amt_con.get_all_hits()
# start_date = (2018, 1, 5)  # This date is UTC time it seems.
# end_date = (2018, 1, 7)
# recent_hits = filter_hits_by_date(all_hits, start_date, end_date)

# You can change from all_hits to recent_hits below. 
all_assignments = get_assignments(amt_con.connection, hits_to_serialize)
print "Number of assignments to serialize = ", len(all_assignments)
pickle_this(all_assignments, outfile)

In [ ]:
# This version is for the sandbox (fetches all hits.)
# all_hits = amt_con.get_reviewable_hits(detailed=False)
# This is for the aristodatasets account

from mturk_utils.annotation_collection import get_assignments
from mturk_utils.annotation_collection import pickle_this
from mturk_utils.annotation_collection import filter_hits_by_date

barebone_hits = []
for page_num in range(1,750): # on aristodatasets account range before v5_2_round2 was: (1,350)
    print "fetching page ", page_num, " ..."
    barebone_hits += [hit for hit in amt_con.connection.get_reviewable_hits(page_size=100, page_number=page_num)]
print "Now fetching detailed HITs for ", len(barebone_hits), "..."
detailed_hits = [amt_con.connection.get_hit(hit.HITId, response_groups=('Minimal','HITDetail'))[0]
                             for hit in tqdm(barebone_hits)]

# title in v5_1 => Answer simple questions about actions in a sentence
hits_to_serialize = [hit for hit in detailed_hits if str(hit.Title)=='Answer simple questions about actions in a sentence']
print "Number of Hits to serialize = ", len(hits_to_serialize), "\nNow fetching assignments..."

all_assignments = get_assignments(amt_con.connection, hits_to_serialize)
print "Number of assignments to serialize = ", len(all_assignments)
pickle_this(all_assignments, outfile)

In [29]:
from mturk_utils.annotation_collection import get_assignments
from mturk_utils.annotation_collection import pickle_this

all_hits = amt_con.get_all_hits()
# start_date = (2018, 1, 7)
# end_date = (2018, 1, 9)
# recent_hits = filter_hits_by_date(all_hits, start_date, end_date)
recent_assignments = get_assignments(amt_con.connection, all_hits)
print "Number of assignments to serialize = ", len(recent_assignments)
pickle_this(recent_assignments, outfile)

100%|██████████| 2/2 [00:00<00:00,  1.61it/s]

Number of assignments to serialize =  2


In [ ]:
#%%time
# new_4b_assignemnts_10_26 = get_assignments(amt_con.connection, new_4b_hits)

In [ ]:
# pickle_this(new_4b_assignemnts_10_26, 'new_4b_assignemnts_10_26.pkl')

# Interact with workers

## reject assignments and ban workers

In [ ]:
# Note: to find bad workers:
# grep "                                                                               " /Users/nikett/quick/sciencesrl-annotation-afresh/results/batch_v2_8k_partial_v2.tsv | cut -f3| less | sort | uniq -c | sort -rn -k1 | less
# find if the workers are really bad: grep A25UOFK73TIWCT /Users/nikett/quick/sciencesrl-annotation-afresh/results/batch_v2_8k_partial_v2.tsv | cut -f8 | grep -v "^$"| wc -l
# get good assignment ids:
# grep -v APTOOB8S2PLJ4 /Users/nikett/quick/sciencesrl-annotation-afresh/results/batch_v2_8k_partial_v2.tsv | cut -f1| tail -n +2 | sort -u > ~/quick/sciencesrl-annotation-afresh/turker-intermediate/assignments_to_accept_8k_batchv2.txt
# nikett:propbank_original_reformatted nikett$ cut -f3,8,9 /Users/nikett/quick/sciencesrl-annotation-afresh/results/batch_v5_3.tsv| grep -v "#" | sort | uniq -c | sed 's/^ *//g'| grep -v "^[0-9] " | sort -rn -k1 | head -5
# v5_3 scisrl
# Is he really bad turker or should we grace him (look at other sample responses)?
# grep -A1 A33XZFIZU249JA /Users/nikett/quick/sciencesrl-annotation-afresh/results/batch_v5_3__round2.tsv | cut -f3,6-| less
# -notsure: 1543 A22502C5A6AFCX
# +spammer: 721 A8TIL9U6YHG5
# +spammer: 543 A1AXTRCJM88NBE
# Then, reject his assignments
# grep "A33XZFIZU24" /Users/nikett/quick/sciencesrl-annotation-afresh/results/batch_v5_3__round2.tsv | cut -f1 | sort -u > /Users/nikett/quick/sciencesrl-annotation-afresh/turker-intermediate/assignments_to_reject_batch_v5_3__round2.txt

def ban_bad_workers(mturk_connection, worker_ids):
    for worker in worker_ids:
        reason_for_block = """
        Your HITs contained many sentences with action verbs, but they were marked 'no action verbs'
        """
        print('blocking ' + str(worker))
        mturk_connection.block_worker(worker, reason_for_block)

In [ ]:
workers_to_ban = unpickle_this('workers_to_ban_prod_1_4.pkl')

In [ ]:
ban_bad_workers(amt_con.connection, list(workers_to_ban))

In [ ]:
to_reject = unpickle_this('assignments_to_reject_prod_1_4.pkl')

In [ ]:
def reject_assignments(mturk_connection, assignments_to_reject):
    feedback_message = """
    Your HITs contained many frames with valid action questions, but they were all marked empty
    """
    reject_count = len(assignments_to_reject)
    for assignment_id in tqdm(assignments_to_reject):
        try:
            mturk_connection.reject_assignment(assignment_id, feedback_message)
        except boto.mturk.connection.MTurkRequestError:
            print('assignment ' + str(assignment_id) + ' already accepted or rejected')

    return reject_count

In [48]:
# reject_assignments(amt_con.connection, list(to_reject))
# file_to_reject_assignments = '/Users/nikett/quick/sciencesrl-annotation-afresh/turker-intermediate/assignments_to_reject_8k_batchv2.txt'
# file_to_reject_assignments = '/Users/nikett/quick/sciencesrl-annotation-afresh/turker-intermediate/assignments_to_reject_batch_v5_3.txt'
# file_to_reject_assignments = '/Users/nikett/quick/sciencesrl-annotation-afresh/turker-intermediate/first_8k/first_8k_rejected_assignments.txt'
# file_to_reject_assignments = '/Users/nikett/quick/sciencesrl-annotation-afresh/turker-intermediate/assignments_to_reject_batch_v5_3__round2.txt'
file_to_reject_assignments = '/Users/nikett/quick/sciencesrl-annotation-afresh/results/batch_v5_3__round3/rejects.tsv'
print "Rejecting assignments ...", file_to_reject_assignments
e_count = 0

with open(file_to_reject_assignments) as f:
    for a in tqdm(f):
        try:
            #print "Rejecting assignment", a
            amt_con.connection.reject_assignment(a)
        except boto.mturk.connection.MTurkRequestError as e:
            print e
            e_count += 1
print str(e_count) + " exceptions"

0it [00:00, ?it/s]

Rejecting assignments ... /Users/nikett/quick/sciencesrl-annotation-afresh/results/batch_v5_3__round3/rejects.tsv


1092it [02:39,  7.36it/s]

0 exceptions


In [ ]:
def pay_bonuses(bonuses_to_pay):
    total_payed = 0
    bonus_reason_template = 'For Flinstones character labels written.'
    for aid, vals in tqdm(bonuses_to_pay.items()):
        n_chars = vals['count']
        wid = vals['worker_id']
        bonus_ammount = boto.mturk.price.Price(0.01 * n_chars)
        total_payed += bonus_ammount.amount
        bonus_reason = bonus_reason_template
        amt_con.connection.grant_bonus(wid, aid, bonus_ammount, bonus_reason)
    return total_payed

In [ ]:
# bonuses = unpickle_this('bonuses_to_pay_3.pkl')

In [ ]:
# pay_bonuses(bonuses)

# Accepting and deleting HITs

Uncomment only when ready to accept or delete hits

reject assignments carefully

In [ ]:
from mturk_utils.annotation_collection import un_pickle_this
accept_results = un_pickle_this(outfile)

In [ ]:
e_count = 0
print "Approving ", len(accept_results), "assignments ..."
for assignment_threeturkers in tqdm(accept_results):
    try:
        for a in accept_results[assignment_threeturkers]:
            amt_con.approve_assignment(assignment=a)
    except boto.mturk.connection.MTurkRequestError as e:
        print e
        e_count += 1
print str(e_count) + " exceptions"

disable hits

In [ ]:
# _ = [amt_con.disable_hit(hit) for hit in tqdm(all_hits)]

hit_id = "36GJS3V78VQDPQD80WH8APBE2DFGJF"
result = amt_con.connection.get_assignments(hit_id)

assignment = result[0]
worker_id = assignment.WorkerId
for answer in assignment.answers[0]:
    # TODO change this.
  if answer.qid == 'answer':
    worker_answer = answer.fields[0]

print "The Worker with ID {} and gave the answer {}".format(worker_id, worker_answer)

deletes hits

In [ ]:
if sandbox:
    print "Deleting", len(amt_con.get_all_hits()), " HITs..."
    amt_con.delete_all_hits()
    print "Done deleting."

In [51]:
# approve rejected assignments.
re_count = 0
file_to_reaccept_assignments ='/Users/nikett/quick/sciencesrl-annotation-afresh/results/batch_v5_3__round3/reject_accepted_mercy.tsv'

with open(file_to_reaccept_assignments) as f:
    for a in tqdm(f):
        try:
            amt_con.connection.approve_rejected_assignment(a)
        except boto.mturk.connection.MTurkRequestError as e:
            re_count += 1
print str(re_count) + " exceptions"

188it [01:01,  4.55it/s]

0 exceptions


In [49]:
e_count = 0

# file_to_accept_assignments = '/Users/nikett/quick/sciencesrl-annotation-afresh/turker-intermediate/assignments_to_accept_8k_batchv2.txt'
#file_to_accept_assignments ='/Users/nikett/quick/sciencesrl-annotation-afresh/turker-intermediate/assignments_to_accept_batch_v5_3.txt'
# file_to_accept_assignments ='/Users/nikett/quick/sciencesrl-annotation-afresh/turker-intermediate/first_8k/first_8k_accepted_assignments.txt'
# file_to_accept_assignments ='/Users/nikett/quick/sciencesrl-annotation-afresh/turker-intermediate/assignments_to_accept_batch_v5_3__round2.txt'
file_to_accept_assignments ='/Users/nikett/quick/sciencesrl-annotation-afresh/results/batch_v5_3__round3/accepts.tsv'
print "Approving assignments ...", file_to_accept_assignments

with open(file_to_accept_assignments) as f:
    for a in tqdm(f):
        try:
            #print "Acceping assignment", a
            amt_con.connection.approve_assignment(a)
        except boto.mturk.connection.MTurkRequestError as e:
#             print e
            e_count += 1
print str(e_count) + " exceptions"

1it [00:00,  7.37it/s]

Approving assignments ... /Users/nikett/quick/sciencesrl-annotation-afresh/results/batch_v5_3__round3/accepts.tsv


3589it [11:34,  6.58it/s]

1 exceptions
